In [5]:
import struct
import numpy as np
from collections import namedtuple

class readBin:

    # constructor
    def __init__(self, path):
        self._path = path
        self._file = None
        self._status = None
        self._time = 0

        # Defining struct datatypes #The character: < means little endian
        self._startbyte_struct = struct.Struct('<II')
        self._frameToStorage_struct = struct.Struct('<iI30s')
        self._frameHeader_struct = struct.Struct("<5i4fI88s")
        self._image_struct = struct.Struct("<19200h")

        # Naming struct variables
        self._startbyte_t = namedtuple('starbyte_t', 'startbyte length')
        self._frameToStorage_t = namedtuple('frameToStorage_t','transmissionType timeStamp tekst')
        self._frameHeader_t = namedtuple('frameHeader_t','frameType frameNum size hRes vRes hFov vFov mainTemp ledTemp displayAmp pad')

    def openBinFile(self):
        self._file = open(self._path, "rb");

    def closeBinFile(self):
        self._file.close();

    def _checkStartbyte(self):
        # check startbyte
        mainheader = self._file.read(self._startbyte_struct.size)
        if len(mainheader) < self._startbyte_struct.size:
            return False
        mainheader = self._startbyte_t._make(self._startbyte_struct.unpack_from(mainheader))

        # if not start byte or wrong length -> find next startbyte
        if mainheader.startbyte != 0xaaaaaaaa or mainheader.length < 0x96ae:
            print("corrupted file. looking for header")

            # iterate through and find next header
            temp = self._file.read(1)
            while temp != b'\xaa\xaa\xaa\xaa':
                temp = self._file.read(1)

                # if 0xaa found check the rest
                if temp == b'\xaa':
                    if self._file.read(3) == b'\xaa\xaa\xaa':
                        temp = b'\xaa\xaa\xaa\xaa'

                # return false if eof
                if temp == '':
                    return False

            # got out of while loop -> found the header
            # rewind file pointer back before the header then recall this function
            print("header found.")
            self._file.seek(self._file.tell()-4)
            self._checkStartbyte()

        # startbyte found and everything fine.
        return True



    def getFrame(self):
        if self._checkStartbyte() == False:
            return None

        # Read in storage header and update timestamp and status
        frameToStorage = self._file.read(self._frameToStorage_struct.size)

        # check if we have eof
        if( len(frameToStorage) < self._frameToStorage_struct.size ) :
            return None

        frameToStorage = self._frameToStorage_t._make(self._frameToStorage_struct.unpack_from(frameToStorage))
        self._time = frameToStorage.timeStamp
        self._status = frameToStorage.tekst.decode("utf-8").strip('\x00')

        frameHeader = self._file.read(self._frameHeader_struct.size)
        rawImage = self._file.read(self._image_struct.size)

        return np.asarray(self._image_struct.unpack_from(rawImage)).reshape((120,160))

    def getStatus(self):
        return self._status

    def getTimeStamp(self):
        return self._time




In [6]:
import cv2
#k = readBin('/Users/gbmello/Dropbox/OsloMet AssistedHome/181101_PLACE_PROJECTNAME/1_data_source/10_7_1.rmt')
k = readBin('/Users/gbmello/Dropbox/OsloMet AssistedHome/181101_PLACE_PROJECTNAME/1_data_source/rm0010/12_7_13.rmt')
#k = readBin('/Volumes/OsloMet/rm0009/10_7_7.rmt')
k.openBinFile()


In [7]:
from datetime import datetime

while(True):
    # Capture frame-by-frame
    frame = k.getFrame()
    frame = (frame/(2**13))
    #frame = (frame/(np.max(frame))) 
    res = cv2.resize(frame,None,fx=3, fy=3, interpolation = cv2.INTER_CUBIC)
    #frame.dtype = np.dtype('uint8')
    # Our operations on the frame come here
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ts = k.getTimeStamp()

    
    # Write some Text
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    positionOfText = (15,15)
    fontScale              = 0.3
    fontColor              = (255,255,255)
    lineType               = 1
    txt = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    cv2.putText(res,txt, 
        positionOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)
    
    
    # Display the resulting frame
    cv2.imshow('frame',res)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything done, release the capture
cv2.destroyAllWindows()

In [ ]:
#Required libraries
from tkinter import filedialog
from tkinter import *

def quit(self):
    self.root.destroy()
    
# starts the root Tk() object
root = Tk()
def open_filename(root):
    """
    Defineds the function that opens the dialog for filetypes
    """
    root.filename =  filedialog.askopenfilename(initialdir = "/",
                                                title = "Select file",
                                                filetypes = (("rmt files","*.jpg"),("hdf5 files","*.h5"),("all files","*.*")))
    print (root.filename)

# create a toplevel menu
menubar = Menu(root)
menubar.add_command(label="Hello!", command=lambda:open_filename(root))
menubar.add_command(label="Quit!", command=root.destroy)

# display the menu
root.config(menu=menubar)

# call the directory dialog box and return the value to a gui property 
#Button(root, text="Open File", command= lambda:open_filename(root)).pack()
#Button(root, text="Quit", command=root.destroy).pack()


#print (root.directory)
root.mainloop()